In [ ]:

    
what_list={
    
    'Latency of the center fixations relative to fixation_target_onset (ms)(median of all fixations per trial) (mean of trials per subject)':
    'CENTER_FIXATION_LATENCY',
    
    'Distance (in degrees) from screen center during fixation period (median of all fixations per trial) (mean of trials per subject)':
    'CENTER_FIXATION_DISTANCE',
    
    'Latency of the first saccade after target onset regardless of its direction (ms)(mean latency of all trials per subject)':
    'LATENCY_FIRST_SACCADE',
    
    'Latency of the first correct saccade after target onset (ms)(mean latency of all trials per subject)':
    'LATENCY_CORRECT_SACCADE',
    
    'Latency of the corrected saccade after wrong saccades (ms)(mean latency of all trials per subject)':    
    'LATENCY_CORRECTED_SACCADE',
    
    'Latency of the correct or corrected saccades (ms)(mean latency of all trials per subject)':    
    'LATENCY_ANY_CORRECT_SACCADE',
    
    'Latency of the first wrong saccade after target onset (ms)(mean latency of all trials per subject)':        
    'LATENCY_FIRST_WRONG_SACCADE',
    
    'Amplitude of the correct saccade (degrees) (mean of all trials per subject)':
    'AMPLITUDE_CORRECT_SACCADE',
    
    'Amplitude of the corrected saccade (degrees) (mean of all trials per subject)':
    'AMPLITUDE_CORRECTED_SACCADE', 
    
    'Amplitude of the wrong saccade (degrees) (mean of all trials per subject)':
    'AMPLITUDE_FIRST_WRONG_SACCADE', 

    'How far the gaze was from where it should be (target or opposite position) after correct or corrected saccades (degrees) (mean of all trials per subject)':
    'FIXATION_OFFSET_TO_TARGET',
    
    
    'Total number of trials where center fixation was not accepted (too far from the center) (percentage)':
    'CENTER_FIXATION_MISSED',
    
    'Total number of successful trials (correct or corrected saccades made) (percentage)':
    'TRIAL_SUCCESSFUL',
    
    'Total number of trials with corrected saccade (percentage)':
    'TRIAL_CORRECTED_ONLY',
    
    'Total number of trials with the first saccade registered as correct (percentage)':
    'TRIAL_CORRECT_ONLY',
    
    'Total number of failed trials (no successful saccade) (percentage)':
    'TRIAL_FAILED',
    
}


which_condition={}
which_display={}

# SaveResults

In [ ]:


def SaveResults():
    global EXCEL_SAVED,RESULTS,what_list,which_condition,out_analyse,RESULTS_FOLDER
    results_to_export=RESULTS.copy(deep=True)
    EXCEL_SAVED=False
    
#     ## create main folder
#     if  os.path.exists(RESULTS_FOLDER)==False:
#         os.makedirs(RESULTS_FOLDER) 

    
    result_folder=RESULTS_FOLDER + '/window %s/'%str(WinowSlider.value)
    if  os.path.exists(result_folder)==False:
        os.makedirs(result_folder) 

    ## save original results for the next time (no need to reanalyze for this window)
    ## TODO: removed because wasn't completed. adding or removing subjects requires Re-analysing.
    #  SaveTable(RESULTS,'RESULTS',path=result_folder)
        
    ## exclude some columns before export
    results_to_export.drop(['CENTER_FIXATION_LATENCY_MEDIAN',
   'CENTER_FIXATION_DISTANCE_TO_CENTER_MEDIAN',
   'CENTER_FIXATION_OFFSET_X_MEDIAN',
   'CENTER_FIXATION_OFFSET_Y_MEDIAN'], axis=1, inplace=True)

    if  os.path.exists(result_folder+ 'Excel')==False:
        os.makedirs(result_folder+ 'Excel') 




    if  os.path.exists(result_folder+ 'CSV')==False:
        os.makedirs(result_folder+ 'CSV') 

    results_to_export.to_csv(result_folder  + 'CSV/FULL_RESULTS.csv',sep=',', index=False, header=True)
    cond_name='D1'
    for k,v in what_list.items():
        for k3,v3 in which_display.items():

            if v3=='Display2':
                cond_name='D2'
            if v3=='Display2_with_display1_successful':
                cond_name='D2 (D1 failed excluded)'

            for k2,v2 in which_condition.items():
                cond_name2=cond_name
                if v2=='CONDITION':
                    cond_name2=cond_name+ '(%s)'% k2
                if v2=='CONDITION_APPEARANCE':
                    cond_name2=cond_name+'(%s)'% k2


                [df,figure_path]=plotResults_step1(v,v2,v3,title='',ylabel=v,SHOW=False,return_output=True)
                fname=result_folder + 'CSV/%s.csv'%( '%s_%s'%(v,cond_name2))
                df.to_csv(fname,sep=',', index=False, header=True)

    with out_analyse:
        print('Results saved in %s'%(result_folder + 'CSV'))

    


    try:
        import openpyxl as openpyxl
        from openpyxl.utils.dataframe import dataframe_to_rows
        from openpyxl.styles import PatternFill
    except:
        print('Exporting results in csv format')
        print('To get results in excel format, install openpyxl via: pip3 install openpyxl')
        return


        



    fname=result_folder + 'Excel/FULL_RESULTS.xlsx'
    ## creating excel file with full results
    with pd.ExcelWriter (fname) as writer:
        results_to_export.to_excel(writer, sheet_name = 'Full',engine='xlsxwriter',index=False)


    for k,v in what_list.items():
        fname=result_folder + 'Excel/%s.xlsx'%(v)
        
        ## sheet name cannot be too long!
        sheet_name='D1'#%list(what_list.values()).index(v)

        wb = openpyxl.Workbook()
        wb.create_sheet(sheet_name)
        del wb['Sheet']
        wb.save(fname)
        wb=openpyxl.load_workbook(fname)



        for k3,v3 in which_display.items():

            if v3=='Display2':
                sheet_name='D2'
            if v3=='Display2_with_display1_successful':
                sheet_name='D2 (no D1 failed)'


                
            for k2,v2 in which_condition.items():

                if v2=='CONDITION':
                    sheet_name=sheet_name+' (conds)'
                if v2=='CONDITION_APPEARANCE':
                    sheet_name=sheet_name+' (types)'


                [df,figure_path]=plotResults_step1(v,v2,v3,title='',ylabel=v,SHOW=False,return_output=True)



                if sheet_name not in wb:
                    wb.create_sheet(sheet_name)


    #                 del wb[wb.active]

    #                 wb.create_sheet(sheet_name)

                sheet = wb[sheet_name]

                Fillcolor = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
                sheet['A1'] = k
                sheet['A1'].fill = Fillcolor

                for r in dataframe_to_rows(df, index=False, header=True):
                    sheet.append(r)


                sheet.merge_cells(start_row=1,start_column=1,end_row=1,end_column=len(df.columns.values))


                for row in sheet[sheet.cell(row=1, column=len(df.columns.values)).coordinate:sheet.cell(row=len(df)+2, column=len(df.columns.values)).coordinate]:
                    for cell in row:
                        cell.fill = Fillcolor

                try:
    #                 if 1:
                    if figure_path!='':
                        img = openpyxl.drawing.image.Image(figure_path)
                        sheet.add_image(img,sheet.cell(row=1, column=len(df.columns.values)+2).coordinate)

                except:
                    # You must install PIL to fetch image objects
                    # pip3 install Pillow
                    pass

        wb.save(fname)

    EXCEL_SAVED=True
    with out_analyse:
        print('Results saved in %s'%(result_folder + 'Excel'))




In [ ]:

def SaveResults_all_in_one():
    global EXCEL_SAVED,RESULTS,what_list,condition_list,out_analyse
    results_to_export=RESULTS.copy(deep=True)
    EXCEL_SAVED=False
    
    if  os.path.exists(RESULTS_FOLDER)==False:
        os.makedirs(RESULTS_FOLDER) 

    ## exclude some columns before export
    results_to_export.drop(['CENTER_FIXATION_LATENCY_MEDIAN',
    'CENTER_FIXATION_DISTANCE_TO_CENTER_MEDIAN',
   'CENTER_FIXATION_OFFSET_X_MEDIAN',
   'CENTER_FIXATION_OFFSET_Y_MEDIAN'], axis=1, inplace=True)


    try:
#     if 1:

        if  os.path.exists(RESULTS_FOLDER+ '/Excel')==False:
            os.makedirs(RESULTS_FOLDER+ '/Excel') 


        fname=RESULTS_FOLDER + '/Excel/RESULTS_%s.xlsx'%(  '_window' + str(WinowSlider.value))
    
        import openpyxl as openpyxl
        from openpyxl.utils.dataframe import dataframe_to_rows
        from openpyxl.styles import PatternFill


        ## creating excel file with full results
        with pd.ExcelWriter (fname) as writer:
            results_to_export.to_excel(writer, sheet_name = 'Full',engine='xlsxwriter',index=False)

        ## loading excel file and adding individual tests
        wb=openpyxl.load_workbook(fname)


        for k,v in what_list.items():
            for k2,v2 in condition_list.items():

  
                ## sheet name cannot be too long!
                sheet_name='results_%s'%list(what_list.values()).index(v)
                if v2=='CONDITION':
                    sheet_name=sheet_name+'_conditions'
                if v2=='CONDITION_APPEARANCE':
                    sheet_name=sheet_name+'_appearance_conditions'

                [df,figure_path]=plotResults_step1(v2,v,title='',ylabel=v,SHOW=False,return_output=True)

                
                wb.create_sheet(sheet_name)
                sheet = wb[sheet_name]

                Fillcolor = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
                sheet['A1'] = k
                sheet['A1'].fill = Fillcolor

                for r in dataframe_to_rows(df, index=False, header=True):
                    sheet.append(r)


                sheet.merge_cells(start_row=1,start_column=1,end_row=1,end_column=len(df.columns.values))


                for row in sheet[sheet.cell(row=1, column=len(df.columns.values)).coordinate:sheet.cell(row=len(df)+2, column=len(df.columns.values)).coordinate]:
                    for cell in row:
                        cell.fill = Fillcolor

                try:
#                 if 1:
                    if figure_path!='':
                        img = openpyxl.drawing.image.Image(figure_path)
                        sheet.add_image(img,sheet.cell(row=1, column=len(df.columns.values)+2).coordinate)

                except:
                    # You must install PIL to fetch image objects
                    # pip3 install Pillow
                    pass

        wb.save(fname)
        EXCEL_SAVED=True
        with out_analyse:
            print('Results saved in %s'%fname)
    
    except:
        if  os.path.exists(RESULTS_FOLDER+ '/CSV')==False:
            os.makedirs(RESULTS_FOLDER+ '/CSV') 

        print('To get results in excel format, install openpyxl via: pip3 install openpyxl')
        results_to_export.to_csv(RESULTS_FOLDER  + '/CSV/RESULTS.csv',sep=',', index=False, header=True)
        for k,v in what_list.items():
            for k2,v2 in condition_list.items(): 
                [df,figure_path]=plotResults_step1(v2,v,title='',ylabel=v,SHOW=False,return_output=True)

                df.to_csv('%s.csv'%figure_path[:-4].replace('Figures','CSV'),sep=',', index=False, header=True)
        with out_analyse:
            print('Results saved in %s'%(RESULTS_FOLDER  + '/CSV'))


In [ ]:
def AddToResultExcelFile(data,sheet_name='',info='',figure_path=''):
    '''
    this will delete all the images that already exist in the excel file
    '''
    if  os.path.exists(RESULTS_FOLDER)==False:
        os.makedirs(RESULTS_FOLDER) 

    fname=RESULTS_FOLDER + '/RESULTS.xlsx'
    try:
        wb=openpyxl.load_workbook(fname)
    except:
        print('RESULTS.xlsx not found')
        return 0
            

    if sheet_name in wb:
        del wb[sheet_name]
#         sheet = wb[sheet_name]

    wb.create_sheet(sheet_name)
    sheet = wb[sheet_name]

        
#     # set value for cell A1=1
    Fillcolor = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
    sheet['A1'] = info
    sheet['A1'].fill = Fillcolor
#     # set value for cell B2=2
#     sheet.cell(row=2, column=2).value = 2

    for r in dataframe_to_rows(data, index=False, header=True):
        sheet.append(r)
        

    sheet.merge_cells(start_row=1,start_column=1,end_row=1,end_column=len(data.columns.values))

    
    for row in sheet[sheet.cell(row=1, column=len(data.columns.values)).coordinate:sheet.cell(row=len(data)+2, column=len(data.columns.values)).coordinate]:
        for cell in row:
            cell.fill = Fillcolor
    
#     try:
    if 1:
        if figure_path!='':
            img = openpyxl.drawing.image.Image(figure_path)
#             img.anchor(sheet.cell(column=len(data.columns.values)+1,row=1))
            print('A%s'%(len(data.columns.values)+1))
            sheet.add_image(img,sheet.cell(row=1, column=len(data.columns.values)+2).coordinate)

#     except:
#         # You must install PIL to fetch image objects
#         # pip3 install Pillow
#         pass
    
    wb.save(fname)
        

# Plot Results

In [ ]:
def plotResults_step2(df,val=None,by=None,hue=None,title=None,ylabel=None,agg_per_subject=None,fname='',SHOW=True):

    
    
    mpld3.disable_notebook()
    figBox, axes = plt.subplots( )
    
    try:
        sns.boxplot(x=by, y=val,data=df,hue=hue,
                 palette="PRGn",  linewidth=1,fliersize=0)
        ax=sns.stripplot(x=by, y=val,data=df,hue=hue,
                  palette="PRGn",  split=True, jitter=True, facecolors="none", alpha=1.0, linewidth=1.0, label='_nolegend_')
    except:
        print('plot error')

    
    if ylabel!=None:
        axes.set_ylabel(ylabel, fontsize=14)
    # if ylim!=None:
    #     axes.set_ylim(ylim[0], ylim[1])
    # figBox.suptitle(main_title)
#     figBox.canvas.set_window_title( ' (box plot)')
    if title!=None:
        axes.set_title(title)
        
#     axes.set_xlabel("")

    plt.gca().xaxis.grid(False)
    plt.gca().yaxis.grid(True)
    
    for label in axes.get_xmajorticklabels() :
        if len(df[by].unique())>4:
            label.set_rotation(90)
        label.set_horizontalalignment("right")
        label.set_fontsize(12)

        
    if hue!=None:
        handles, labels = axes.get_legend_handles_labels()
        # When creating the legend, only use the first two elements
        # to effectively remove the last two.
        l = plt.legend(handles[0:len(df[hue].unique())], labels[0:len(df[hue].unique())], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    
    if  os.path.exists(RESULTS_FOLDER)==False:
        os.makedirs(RESULTS_FOLDER) 

    plt.tight_layout()
    if fname!='':
        plt.savefig( fname, dpi=150 )

    if SHOW:
        plt.show()
    else:
        plt.close()

def plotResults_step1(what,condition,for_which_display,title='',ylabel= '',SHOW=False,return_output=False):


    if type(RESULTS)==type(None):
        print('You need to analyse the data first')
        return [None,None]
    

    if  os.path.exists(RESULTS_FOLDER + '/window %s/Figures'%str(WinowSlider.value) )==False:
        os.makedirs(RESULTS_FOLDER + '/window %s/Figures'%str(WinowSlider.value) ) 



    groupby_list=['SUBJECT_NAME','SUBJECT_GROUP']
    if condition=='CONDITION':
        groupby_list=groupby_list+['CONDITION']

    if condition=='CONDITION_APPEARANCE':
        groupby_list=groupby_list+['CONDITION_APPEARANCE']


    ## here we decide whether we need to exclude trials with center fixation missing
    ## I make to versions of the result and use the filtered or unfiltered table depending on 'what' AND CenterFixMiss_Exclude_Checkbox.value

    RESULTS_sub=RESULTS.copy(deep=True)
    RESULTS_sub.loc[RESULTS_sub['CONDITION_APPEARANCE'].str.contains('^TD'),'CONDITION_APPEARANCE']='TD'
    
    RESULTS_sub['correct_or_corrected']=RESULTS_sub['LATENCY_CORRECTED_SACCADE'].fillna(RESULTS_sub['LATENCY_CORRECT_SACCADE'])
    
   
    ## RD
    if for_which_display=='Display1':
        RESULTS_sub=RESULTS_sub.query('SECOND_DISPLAY==0')
    elif for_which_display=='Display2':
        RESULTS_sub=RESULTS_sub.query('SECOND_DISPLAY==1')
    elif for_which_display=='Display2_with_display1_successful':
        RESULTS_sub=RESULTS_sub.query('SECOND_DISPLAY==1 and DISPLAY_1_WAS_SUCCESSFUL==1')
    
    

    
    

    
    df_missing_fix_not_excluded=RESULTS_sub.groupby(groupby_list).agg(['mean','sum','count','first']).reset_index()
    df_missing_fix_not_excluded.columns = ['_'.join(tup).rstrip('_') for tup in df_missing_fix_not_excluded.columns.values]


    
    df_missing_fix_excluded=RESULTS_sub.query('CENTER_FIXATION_ACCEPTED==1').groupby(groupby_list).agg(['mean','sum','count','first']).reset_index()
    df_missing_fix_excluded.columns = ['_'.join(tup).rstrip('_') for tup in df_missing_fix_excluded.columns.values]

 

    
    ## don't exclude if checkbox is not ticked
    if CenterFixMiss_Exclude_Checkbox.value==True:
        df_missing_fix_excluded=df_missing_fix_not_excluded
    
    columns_included_in_the_report=groupby_list
    
    
    
    df=pd.DataFrame()
    
    ## calculate result value based on 'what'
    if what=='CENTER_FIXATION_LATENCY':
        df=df_missing_fix_excluded
        df['VALUE']=df['CENTER_FIXATION_LATENCY_REL_FIXATION_ONSET_MEDIAN_mean']
        
    if what=='CENTER_FIXATION_DISTANCE':
        df=df_missing_fix_not_excluded
        df['VALUE']=df['CENTER_FIXATION_DISTANCE_TO_CENTER_DEG_MEDIAN_mean']
        
    if what=='LATENCY_FIRST_SACCADE':
        df=df_missing_fix_excluded
        df['VALUE']=df['LATENCY_FIRST_SACCADE_mean']
        
    if what=='LATENCY_CORRECT_SACCADE':
        df=df_missing_fix_excluded
        df['VALUE']=df['LATENCY_CORRECT_SACCADE_mean']
        
    if what=='LATENCY_CORRECTED_SACCADE':
        df=df_missing_fix_excluded
        df['VALUE']=df['LATENCY_CORRECTED_SACCADE_mean']
        
    if what=='LATENCY_ANY_CORRECT_SACCADE':
        df=df_missing_fix_excluded
        df['VALUE']=df['correct_or_corrected_mean']
        
    if what=='LATENCY_FIRST_WRONG_SACCADE':
        df=df_missing_fix_excluded
        df['VALUE']=df['LATENCY_FIRST_WRONG_SACCADE_mean']

    if what=='AMPLITUDE_CORRECT_SACCADE':
        df=df_missing_fix_excluded
        df['VALUE']=df['AMPLITUDE_CORRECT_SACCADE_mean']
        
    if what=='AMPLITUDE_CORRECTED_SACCADE':
        df=df_missing_fix_excluded
        df['VALUE']=df['AMPLITUDE_CORRECTED_SACCADE_mean']
        
    if what=='AMPLITUDE_FIRST_WRONG_SACCADE':
        df=df_missing_fix_excluded
        df['VALUE']=df['AMPLITUDE_FIRST_WRONG_SACCADE_mean']
        
    if what=='FIXATION_OFFSET_TO_TARGET':
        df=df_missing_fix_excluded
        df['VALUE']=df['FIXATION_OFFSET_TO_TARGET_mean']

        
    ## results are made from non-excluded part of the fixationsaccade table. This means that all the trials that exist in the results table are those that were valid. so we discard the total number of trials 
    ## in the fixationsaccade table and take the total number of trials from the result table. 
    ## We don't care about blocks here. we take the mean across all included blocks (pretending that all trials are in one block).## trial index in each block may start from 1. 
    ## total number of condition 'Display2_with_display1_successful' is the total number excluding failed trials in D1.
    
    df['TOTAL_NUMBER_OF_TRIALS']=np.nan
  
    
    if what=='CENTER_FIXATION_MISSED':
        df=df_missing_fix_not_excluded

        df['TOTAL_NUMBER_OF_TRIALS']=df['TRIAL_INDEX_count']/len(RESULTS_sub['SECOND_DISPLAY'].unique())
 
        df['NUMBER_OF_TRIALS_WITH_CENTER_FIXATION_MISSING']=df['TOTAL_NUMBER_OF_TRIALS']-df['CENTER_FIXATION_ACCEPTED_sum']
              
        df['VALUE']=(df['NUMBER_OF_TRIALS_WITH_CENTER_FIXATION_MISSING']/df['TOTAL_NUMBER_OF_TRIALS'])*100

        columns_included_in_the_report.append('NUMBER_OF_TRIALS_WITH_CENTER_FIXATION_MISSING')
        columns_included_in_the_report.append('TOTAL_NUMBER_OF_TRIALS')
 
    if what=='TRIAL_SUCCESSFUL':
        df=df_missing_fix_excluded
        
        df['TOTAL_NUMBER_OF_TRIALS']=df['TRIAL_INDEX_count']/len(RESULTS_sub['SECOND_DISPLAY'].unique())

        df['NUMBER_OF_SUCCESSFUL_TRIALS']=df['TRIAL_SUCCESSFUL_sum']
        
        df['VALUE']=(df['NUMBER_OF_SUCCESSFUL_TRIALS']/df['TOTAL_NUMBER_OF_TRIALS'])*100

        columns_included_in_the_report.append('NUMBER_OF_SUCCESSFUL_TRIALS')
        columns_included_in_the_report.append('TOTAL_NUMBER_OF_TRIALS')


    if what=='TRIAL_CORRECTED_ONLY':
        df=df_missing_fix_excluded
        
        df['TOTAL_NUMBER_OF_TRIALS']=df['TRIAL_INDEX_count']/len(RESULTS_sub['SECOND_DISPLAY'].unique())

        df['NUMBER_OF_CORRECTED_TRIALS']=df['LATENCY_CORRECTED_SACCADE_count']

        df['VALUE']=(df['NUMBER_OF_CORRECTED_TRIALS']/df['TOTAL_NUMBER_OF_TRIALS'])*100

        columns_included_in_the_report.append('NUMBER_OF_CORRECTED_TRIALS')
        columns_included_in_the_report.append('TOTAL_NUMBER_OF_TRIALS')



    if what=='TRIAL_CORRECT_ONLY':
        df=df_missing_fix_excluded
        
        df['TOTAL_NUMBER_OF_TRIALS']=df['TRIAL_INDEX_count']/len(RESULTS_sub['SECOND_DISPLAY'].unique())

        df['NUMBER_OF_CORRECT_TRIALS']=df['LATENCY_CORRECT_SACCADE_count']

        df['VALUE']=(df['NUMBER_OF_CORRECT_TRIALS']/df['TOTAL_NUMBER_OF_TRIALS'])*100

        columns_included_in_the_report.append('NUMBER_OF_CORRECT_TRIALS')
        columns_included_in_the_report.append('TOTAL_NUMBER_OF_TRIALS')

        
    if what=='TRIAL_FAILED':
        df=df_missing_fix_excluded
        
        df['TOTAL_NUMBER_OF_TRIALS']=df['TRIAL_INDEX_count']/len(RESULTS_sub['SECOND_DISPLAY'].unique())

        df['NUMBER_OF_FAILED_TRIALS']=df['TOTAL_NUMBER_OF_TRIALS']-df['TRIAL_SUCCESSFUL_sum']

        df['VALUE']=(df['NUMBER_OF_FAILED_TRIALS']/df['TOTAL_NUMBER_OF_TRIALS'])*100

        columns_included_in_the_report.append('NUMBER_OF_FAILED_TRIALS')
        columns_included_in_the_report.append('TOTAL_NUMBER_OF_TRIALS')


        
        


    for col in df.columns.values:
        if col.startswith('SUBJECT_') and col[-6:]=='_first':
            columns_included_in_the_report.append(col)

    df=df.loc[:,columns_included_in_the_report+['VALUE']]
    
    for col in df.columns.values:
        if col.endswith('_first'):
            df=df.rename(columns={col:col[:-6]})
    
    
    
    ## sort df by value
    df=df.sort_values(by='VALUE', ascending=False)
    

    name_excel_file=RESULTS_FOLDER + '/window %s/'%str(WinowSlider.value) + 'Excel/%s.xlsx'%(what)
    ## figure name
    cond_name='D1'
    if for_which_display=='Display2':
        cond_name='D2'
    if for_which_display=='Display2_with_display1_successful':
        cond_name='D2 (D1 failed excluded)'

    if condition=='CONDITION':
        cond_name=cond_name+' (conditions)'
    if condition=='CONDITION_APPEARANCE':
        cond_name=cond_name+' (target position)'
        
    figure_path=RESULTS_FOLDER + '/window %s/Figures'%str(WinowSlider.value)  + '/%s.png'%(  '%s_%s'%(what,cond_name))
    name_csv_file=RESULTS_FOLDER + '/window %s/CSV'%str(WinowSlider.value) + '/%s.csv'%(  '%s_%s'%(what,cond_name))


    if SHOW:
        if EXCEL_SAVED:
            print('results saved in sheet %s in %s'%(cond_name,name_excel_file))
        else:
            print('results saved in %s'%(name_csv_file))
            
            

    if condition=='All':
        plotResults_step2(df,val='VALUE',by="SUBJECT_GROUP",title=title,ylabel=ylabel,fname=figure_path,SHOW=SHOW)
#     ## ??????????????????? did removing excule affect any of the aggregated results?!
    if condition=='CONDITION':
        plotResults_step2(df,val='VALUE',by="SUBJECT_GROUP",hue='CONDITION',title=title,ylabel=ylabel,fname=figure_path,SHOW=SHOW)

#     ## ??????????????????? did removing excule affect any of the aggregated results?!
    if condition=='CONDITION_APPEARANCE':
        plotResults_step2(df,val='VALUE',by="SUBJECT_GROUP",hue='CONDITION_APPEARANCE',title=title,ylabel=ylabel,fname=figure_path,SHOW=SHOW)

    if return_output:
        return [df,figure_path]











# Display_analysis_UI

In [ ]:
RESULTS=None
out_analyse=None
WinowSlider=None
label_slider=None
AnalyseButton=None
AnalyseProgress=None
UI_analyse=None
CenterFixMiss_Exclude_Checkbox=None
# full list of widget attributes: https://github.com/jupyter-widgets/ipywidgets/blob/9e77366ae37239f9e66857473f3896d0fe37bf5f/packages/schema/jupyterwidgetmodels.latest.md

def update_slider_window(change):
    with out_analyse:
        clear_output()
    if change['type'] == 'change' and change['name'] == 'value':
        label_slider.value='(%s ms,%s ms)'%(change['new'][0],change['new'][1])
        Button_Refresh_Refresh_Needed()
        

def Make_Refresh_Button():
    return widgets.Button(
                description='Analyse',
                disabled=False,
                button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click here to apply changes',
                icon='fa-refresh',
                layout=widgets.Layout(flex='1 1 auto', width='auto',height='auto'),
                )

def Button_Refresh_Waiting():
    AnalyseButton.description='Wait!'
    AnalyseButton.button_style='warning'
    AnalyseButton.icon='fa-hourglass-2'

def Button_Refresh_Refresh_Needed():
    AnalyseButton.description='Re-Analyse!'
    AnalyseButton.button_style='warning'
    AnalyseButton.icon='fa-refresh'
def Button_Refresh_Updated():
    AnalyseButton.description='Analysed'
    AnalyseButton.button_style='success'
    AnalyseButton.icon='fa-check'


def button_refresh_clicked(change):
    global RESULTS,which_condition,which_display
    with out_analyse:
        clear_output()
    Button_Refresh_Waiting()
    
    ## check if the results exist
    global RESULTS_FOLDER
    result_folder=RESULTS_FOLDER + '/window %s/'%str(WinowSlider.value)
    _already_analyzed=os.path.isfile(os.path.join(result_folder, 'RESULTS.sm.1'))
    

#     if _already_analyzed:
#         RESULTS=LoadTable('RESULTS',path=result_folder)
#     else:
    RESULTS=AnalyseForWindow(FIXATIONSACCADE_TABLE)
    
    
    ##  make which_condition
    which_condition={
    'All':'All',
    ', '.join( RESULTS['CONDITION_APPEARANCE'].unique()):'CONDITION_APPEARANCE'}
    if len(RESULTS['CONDITION'].unique())>1:
        which_condition.update({', '.join( RESULTS['CONDITION'].unique()):'CONDITION'})

    
    ## make which_display
    which_display={'Only include display 1':'Display1'}

    if len(RESULTS['SECOND_DISPLAY'].unique())>1:# RD
        which_display.update({'Only include display 2':'Display2',
                             'Display 2 excluding those trials where the first display was failed':'Display2_with_display1_successful'})
    
    if _already_analyzed==False:
        AnalyseButton.description='Exporting results'
        SaveResults()   
    Button_Refresh_Updated()




def Display_analysis_UI():
    tab = widgets.Tab()
    global AnalyseButton,WinowSlider,label_slider,AnalyseProgress,UI_analyse,out_analyse,CenterFixMiss_Exclude_Checkbox
    WinowSlider=widgets.IntRangeSlider(
                                value=(80,+800),                                 
                                min=-500,
                                max=1000,
                                step=1,
                                description='Time Window rel Target Onset:',
                                disabled=False,
                                continuous_update=True,
                                orientation='horizontal',
                                readout=False,
                                readout_format='',
        layout=widgets.Layout(display='flex',flex_grow=2,width='80%'),
#                                 layout=widgets.Layout(display='flex',flex_grow=2,width='100%'),
        style={'description_width':  'initial'})
    
    label_slider=widgets.Label(value='(%s ms,%s ms)'%(WinowSlider.value[0],WinowSlider.value[1]) ,
                               layout=widgets.Layout(display='flex',flex_grow=1,width='20%'))
    
    WinowSlider.observe(update_slider_window,names= 'value')

    
    CenterFixMiss_Exclude_Checkbox=widgets.Checkbox(
    value=True,
    description='Exclude trials with center fixation missing',
    disabled=False,style={'description_width': 'initial'})
    
    
    
    AnalyseButton=Make_Refresh_Button()
    AnalyseButton.on_click(button_refresh_clicked)

    AnalyseProgress = widgets.FloatProgress(min=0,value=0, max=1,style={'description_width': 'initial'},layout=widgets.Layout( width='auto'))


    out_analyse = widgets.Output()


    
    

    
    UI_analyse =widgets.VBox(
        children=[ 
            widgets.HBox(
                children=[    
                widgets.VBox(
                    children=[
                        widgets.HBox(
                            children=[WinowSlider,label_slider], layout=widgets.Layout(display='flex',flex_flow='row',align_items='stretch',width='100%')),
                        CenterFixMiss_Exclude_Checkbox],
                    layout=widgets.Layout(display='flex',
                    flex_flow='column',border='solid 1px',
                    align_items='stretch',
                    width='80%')),AnalyseButton], layout=widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%')),AnalyseProgress,out_analyse],layout=widgets.Layout(display='flex',
                    flex_flow='column',border='solid 1px',
                    align_items='stretch',
                    width='100%'))

#     UI_analyse =widgets.VBox(children=[ widgets.HBox(children=[WinowSlider,label_slider,AnalyseButton], layout=box_layout),CenterFixMiss_Exclude_Checkbox,AnalyseProgress],layout=widgets.Layout(display='flex',
#                     flex_flow='column',border='solid 1px',
#                     align_items='stretch',
#                     width='100%'))


    display(UI_analyse)

# AnalyseForWindow()

In [4]:

def AnalyseForWindow(FixationSaccade_table):

    cols=['TRIAL_INDEX','SECOND_DISPLAY','BLOCK', 'CONDITION', 'CONDITION_APPEARANCE']
    for col in FixationSaccade_table.columns.values:
        if col.startswith('SUBJECT_'):
            cols.append(col)

    results=FixationSaccade_table.query('EXCLUDE==False')[cols].groupby(['SUBJECT_NAME','TRIAL_INDEX','SECOND_DISPLAY','BLOCK'])
    AnalyseProgress.max=len(results)
    AnalyseProgress.value=0
    results=results.first()

    window=WinowSlider.value
    
    temp_previous_subtrial=None


    for (NAME,TRIAL,IS_SECOND_DISPLAY,BLOCK),tb_subtrial in FixationSaccade_table.query('EXCLUDE==False').groupby(['SUBJECT_NAME','TRIAL_INDEX','SECOND_DISPLAY','BLOCK']):


        subtrial_temp_dic={}

        AnalyseProgress.value=AnalyseProgress.value+1

                ## *******************  Center Fixation during fixation target
        ## look at the center fixation period of all trials and return offest from center
        ## It help to identify the calibration offset for each individual
        ## offset is either due to calibration issue or user performance
#             time0=time.time()
        center_fixations=tb_subtrial.query(r"EVENT=='fixation' and EVENT_START_INDEX<TARGET_ONSET_INDEX")

        if len(center_fixations)!=0:## if there is any fixation
            center_fixations_dic={}
            center_fixations_dic.update({'CENTER_FIXATION_OFFSET_X':center_fixations['FIXATION_MEAN_X'].values-SCREEN_CENTER[0]})
            center_fixations_dic.update({'CENTER_FIXATION_OFFSET_Y':center_fixations['FIXATION_MEAN_Y'].values-SCREEN_CENTER[1]})
            center_fixations_dic.update({'CENTER_FIXATION_LATENCY':center_fixations['FIXATION_START_TIMESTAMP'].values-center_fixations['TARGET_ONSET_TIMESTAMP'].values})
            center_fixations_dic.update({'CENTER_FIXATION_LATENCY_REL_FIXATION_ONSET':center_fixations['FIXATION_START_TIMESTAMP'].values-center_fixations['FIXATION_TARGET_ONSET_TIMESTAMP'].values})
            center_fixations_dic.update({'CENTER_FIXATION_DISTANCE_TO_CENTER':np.apply_along_axis(np.linalg.norm, 1,list( zip(center_fixations_dic['CENTER_FIXATION_OFFSET_X'],center_fixations_dic['CENTER_FIXATION_OFFSET_Y'])) )  })
            center_fixations_dic.update({'CENTER_FIXATION_DISTANCE_TO_CENTER_DEG':pixels_to_degrees_center_based(center_fixations_dic['CENTER_FIXATION_DISTANCE_TO_CENTER'])  })

            for col,val in center_fixations_dic.items():
                subtrial_temp_dic.update({'%s_MEDIAN'%col:np.nanmedian(val)} )

        subtrial_temp_dic.update( { 'CENTER_FIXATION_ACCEPTED':(np.nanmedian(center_fixations_dic['CENTER_FIXATION_DISTANCE_TO_CENTER_DEG'])<pixels_to_degrees_center_based(EXPERIMENT_SETTINGS['AOI_size']/2)).astype(int)})



        ## *******************  Analyse latency 

        tb_subtrial['saccade_latency']=tb_subtrial['SACCADE_START_TIMESTAMP']-tb_subtrial['TARGET_ONSET_TIMESTAMP']

        saccades_in_window=tb_subtrial.query(r"EVENT=='saccade' and saccade_latency>=%s and saccade_latency<=%s"%(window[0],window[1]))

        ## if no saccade in the window (TODO: not sure if I can just continue)
        if len(saccades_in_window)==0:
            continue 

        # correct saccades
        saccades_correct=saccades_in_window.query(' or '.join( ['%s==1'%s for s in saccades_in_window.columns[(saccades_in_window.columns.str.contains('^SACCADE_TOWARDS_TARGET')) | (saccades_in_window.columns.str.contains('^SACCADE_TOWARDS_OPPOSITE_.*'))] ]))


        # wrong saccades (towards distractors)
        if saccades_in_window.columns.str.contains('^SACCADE_TOWARDS_DISTRACTOR_.*').any():
            saccades_wrong=saccades_in_window.query(' or '.join( ['%s==1'%s for s in saccades_in_window.columns[saccades_in_window.columns.str.contains('^SACCADE_TOWARDS_DISTRACTOR_.*')] ]))
        else:## in PS
            saccades_wrong=saccades_in_window.query(' and '.join( ['%s==0'%s for s in saccades_in_window.columns[(saccades_in_window.columns.str.contains('^SACCADE_TOWARDS_TARGET')) ] ]))


        # Latency of any kind of saccade
        if len(saccades_in_window)!=0:
            subtrial_temp_dic.update({'LATENCY_FIRST_SACCADE':saccades_in_window['saccade_latency'].iloc[0]})

        # successful
        if len(saccades_correct)!=0:
            subtrial_temp_dic.update({'TRIAL_SUCCESSFUL':1})


            if EXPERIMENT_SETTINGS['type']=='Anti-Saccade':
                TARGET_POS=(saccades_correct['DISTRACTOR_1_X'].iloc[0],saccades_correct['DISTRACTOR_1_Y'].iloc[0])
                TARGET_POS=rotatePoint(SCREEN_CENTER,TARGET_POS,180)
            else:
                TARGET_POS=(saccades_correct['TARGET_X'].iloc[0],saccades_correct['TARGET_Y'].iloc[0])

            dist_to_target=pixels_to_degrees_center_based(distance.euclidean( (saccades_correct['SACCADE_B_X'].iloc[0],saccades_correct['SACCADE_B_Y'].iloc[0]) ,TARGET_POS))                  

            subtrial_temp_dic.update({'FIXATION_OFFSET_TO_TARGET':dist_to_target})


#             print('s1 ',time.time()-time0)
#             time0=time.time()
        ## corrected trial
        ## set default in case corrected not found in the dataset!
        for key in ['LATENCY_CORRECTED_SACCADE','AMPLITUDE_CORRECTED_SACCADE','LATENCY_FIRST_WRONG_SACCADE','AMPLITUDE_FIRST_WRONG_SACCADE']:
            subtrial_temp_dic.update({key:np.nan})


        if len(saccades_wrong)!=0 and len(saccades_correct)!=0:
            if saccades_wrong['saccade_latency'].iloc[0] > saccades_correct['saccade_latency'].iloc[0]:
                ## normal correct saccade
                subtrial_temp_dic.update({'LATENCY_CORRECT_SACCADE':saccades_correct['saccade_latency'].iloc[0]})
                subtrial_temp_dic.update({'AMPLITUDE_CORRECT_SACCADE':saccades_correct['SACCADE_AMPLITUDE'].iloc[0]})



            else:
                ## then it's a corrected saccade
                subtrial_temp_dic.update({'LATENCY_FIRST_WRONG_SACCADE':saccades_wrong['saccade_latency'].iloc[0]})
                subtrial_temp_dic.update({'AMPLITUDE_FIRST_WRONG_SACCADE':saccades_wrong['SACCADE_AMPLITUDE'].iloc[0]})
                subtrial_temp_dic.update({'LATENCY_CORRECTED_SACCADE':saccades_correct['saccade_latency'].iloc[0]})
                subtrial_temp_dic.update({'AMPLITUDE_CORRECTED_SACCADE':saccades_correct['SACCADE_AMPLITUDE'].iloc[0]})



        elif len(saccades_wrong)==0 and len(saccades_correct)!=0:
            ## normal correct saccade
            subtrial_temp_dic.update({'LATENCY_CORRECT_SACCADE':saccades_correct['saccade_latency'].iloc[0]})
            subtrial_temp_dic.update({'AMPLITUDE_CORRECT_SACCADE':saccades_correct['SACCADE_AMPLITUDE'].iloc[0]})

        elif len(saccades_wrong)!=0 and len(saccades_correct)==0:
            ## only wrong saccades
            subtrial_temp_dic.update({'LATENCY_FIRST_WRONG_SACCADE':saccades_wrong['saccade_latency'].iloc[0]})
            subtrial_temp_dic.update({'AMPLITUDE_FIRST_WRONG_SACCADE':saccades_wrong['SACCADE_AMPLITUDE'].iloc[0]})

            

        ## check previous display and report its status in this row as well
        subtrial_temp_dic.update({'DISPLAY_1_WAS_SUCCESSFUL':np.nan})
        if IS_SECOND_DISPLAY==1:

            saccades_in_window_1=temp_previous_subtrial.query(r"EVENT=='saccade' and saccade_latency>=%s and saccade_latency<=%s"%(window[0],window[1]))

            saccades_correct_1=saccades_in_window_1.query(' or '.join( ['%s==1'%s for s in saccades_in_window.columns[(saccades_in_window.columns.str.contains('^SACCADE_TOWARDS_TARGET')) | (saccades_in_window.columns.str.contains('^SACCADE_TOWARDS_OPPOSITE_.*'))] ]))

            if len(saccades_correct_1)!=0:
                subtrial_temp_dic.update({'DISPLAY_1_WAS_SUCCESSFUL':1})
            else:
                subtrial_temp_dic.update({'DISPLAY_1_WAS_SUCCESSFUL':0})

#             print('s2 ',time.time()-time0)
#             time0=time.time()
        temp_previous_subtrial=tb_subtrial.copy(deep=True)


        for new_col in subtrial_temp_dic.keys():
            if new_col not in results.columns:
                results[new_col]=np.nan

        results.set_value((NAME,TRIAL,IS_SECOND_DISPLAY,BLOCK),subtrial_temp_dic.keys(),subtrial_temp_dic.values())
#             print('s3 ',time.time()-time0)
#             print('--')
            
    AnalyseProgress.value=0
    results=results.reset_index()
    results.loc[results['TRIAL_SUCCESSFUL'].isnull(),'TRIAL_SUCCESSFUL']=0
    
    return results




